In [1]:
import sys
import os
import muon as mu


# Add the Functions folder to your path
sys.path.append(os.path.abspath("/Users/alexandra/Desktop/EG-CLR/Functions"))

# Now import your functions
from QC_functions import *
from CLR_functions import *

In [19]:
# Load multimodal data
mdata = mu.read("/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/processed_data/multiome_data.h5mu")

adata_gem = mdata['rna']
adata_atac = mdata['atac']

/opt/anaconda3/envs/CLR/lib/python3.11/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/opt/anaconda3/envs/CLR/lib/python3.11/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [21]:
# obtain orginal read counts
adata_org = sc.read_10x_mtx('/Users/alexandra/Desktop/Data/Single-Cell/hg19_10xCloud_aligned_data/filtered_feature_bc_matrix', gex_only = False)

gex_rows = list(map(lambda x: x == 'Gene Expression', adata_org.var['feature_types']))
atac_rows = list(map(lambda x: x == 'Peaks', adata_org.var['feature_types']))

adata_gem_org = adata_org[:, gex_rows].copy()
adata_atac_org = adata_org[:, atac_rows].copy()

adata_gem_org.var_names_make_unique()
adata_atac_org.var_names_make_unique()

In [ ]:
# replace read counts with orgaianl read counts
gem_index = adata_gem.var_names
atac_index = adata_atac.var.index

cell_index = adata_gem.obs.index

adata_gem_new = adata_gem_org[:, adata_gem_org.var_names.isin(gem_index)].copy()
adata_atac_new = adata_atac_org[:, adata_atac_org.var_names.isin(gem_index)].copy()

adata_gem_new = adata_gem_new[adata_gem_new.obs_names.isin(cell_index)].copy()
#adata_atac_new = adata_atac_new[adata_atac_new.obs_names.isin(cell_index)].copy()


In [ ]:
# normalizing scRNA-seq by chromasome 

AnnData object with n_obs × n_vars = 10359 × 13703
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'S_score', 'G2M_score', 'phase'
    var: 'gene_ids', 'feature_types', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'log1p'

In [ ]:
# define promoter, enhancer and gene body regions

In [ ]:
# binarize scATAC-seq 

In [ ]:
# compute CLR matrix for GATA1 -> ChrX